In [1]:
import datasets
from datasets import load_dataset, Dataset
from pathlib import Path
from transformers import GPT2Tokenizer, GPT2Model,  AutoModelForQuestionAnswering, AutoTokenizer
from transformers import Trainer, TrainingArguments

project_dir = Path().cwd().parent
data_dir = project_dir.joinpath("data")

/home/wonseok/.pyenv/versions/3.10.2/envs/nlp/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-11-18 19:49:03.499843: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 19:49:03.598801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 19:49:03.598924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 19:49:03.616004: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to reg

In [2]:
import json

with open('../data/processed/triviaqa_train.json','r') as f :
    data = json.load(f)

In [3]:
def extractor(paragraph) :
    return paragraph['paragraphs'][0]

all_data = [extractor(book) for book in data['data']]

In [5]:
with open('../data/processed/trivia.json', 'w', encoding='utf-8') as f :
    json.dump(all_data, f)
# data['data']

In [6]:
from datasets import Dataset

In [16]:
dataset = load_dataset('json', data_files="../data/processed/trivia.json", split='train[:10000]')
dataset = dataset.train_test_split(test_size=0.2)

Found cached dataset json (/home/wonseok/.cache/huggingface/datasets/json/default-9251c168c2e7bf69/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [57]:
dataset['train']['qas'][0][0]

{'answers': [],
 'id': 'tc_1062--Chemical_element.txt',
 'is_impossible': True,
 'question': 'Which element is named after Pierre and Marie Curie?'}

In [48]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [58]:
def preprocess_function(examples):
    questions = [q[0]['question'].strip() for q in examples["qas"]]
    contexts = [context.strip() for context in examples['context']]
    answers = [q[0]['answers'] for q in examples["qas"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=1024,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        if len(answers[i]) < 1 :
            start_positions.append(0)
            end_positions.append(0)
            continue
        else :
            answer = answers[i][0]

        start_char = answer["answer_start"]
        end_char = answer["answer_start"][0] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx

        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [59]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

IndexError: list index out of range

In [48]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [50]:
training_args = TrainingArguments(
    output_dir="../model/trivia_gpt",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

In [74]:
context = dataset['train'][0]['entity_pages']['wiki_context'][0]

In [135]:
inputs = tokenizer(question, context, return_tensors='pt', return_token_type_ids=True, max_length=1024, truncation=True)

In [136]:
import torch
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.1998, -9.9643, -8.0330,  ..., -6.3425, -5.7444, -6.0709]]), end_logits=tensor([[-0.0966, -0.7796, -0.9979,  ..., -1.6981, -1.8853, -1.9204]]), hidden_states=None, attentions=None)


In [48]:
outputs.start_logits

tensor([[ 2.3874,  6.4358,  5.6925,  5.6773,  5.6681,  5.2134,  5.1008,  5.8855,
          5.3893,  5.8450,  5.3206,  5.4511,  5.6941,  5.4772,  5.1780,  5.5038,
          5.4234,  4.6833,  4.7428,  4.3512,  5.7624,  4.6304,  3.8184,  3.7696,
          4.0553,  3.6392,  4.7540,  4.3875,  3.5800,  3.8620,  3.5720,  3.4407,
          3.1645,  3.3903,  4.7443,  5.5327,  4.2520,  3.7863,  2.0929,  1.4395,
          4.9716,  3.3676,  3.4566,  3.7941,  4.2121,  2.7184,  1.9635,  1.9507,
          4.4011,  2.8790,  2.5880,  2.7406,  0.9453,  3.2911,  2.6771,  3.8936,
          5.1057,  2.8449,  4.4624,  3.8717,  4.5720,  4.3178,  5.1858,  4.0228,
          3.8016,  4.3358,  4.6010,  4.0006,  3.4059,  4.0620,  3.5027,  4.5354,
          4.6953,  5.2006,  5.0762,  3.5061,  4.3643,  4.9156,  4.7615,  4.4697,
          4.1841,  3.9989,  4.2213,  3.8873,  4.2942,  2.3721,  4.0536,  3.2292,
          3.0520,  3.4255,  2.8044,  3.4301,  4.2049,  3.5849,  3.4642,  4.4165,
          5.4953,  5.9342,  